# Handling of sequencing reads

## Required tools

Reads are typically available in [fastq](https://en.wikipedia.org/wiki/FASTQ_format) format after conducting sequencing experiments (Illumina, Nanopore, PacBio).

Our analysis will be done in the **Compute Canada (CC)** cluster of analysis. To login into the cluster, follow the instructions available in the [wiki page for new users](https://docs.alliancecan.ca/wiki/SSH). 

In the **CC cluster**, there are several tools available for analysis and they can each be installed using the command `module load tool/v.XX`.

For this tutorials, tools will be made available using singularity containers, which can be run using the command `singularity run tool_image`. These tools have been made available in the environment already, so there is no need to download them.

Tools used in this tutorial:
- seqkit
- fastqc
- multiqc

We will first explore the structure of our environment and the folders available. It is good practice to assign a directory (`tutorials`) to every project. Inside this main directory, we will create subdirectories with results of every step of our genomic analysis.

In [14]:
# change to home directory and print the PATH to it
cd | pwd -P

# show what is available in home directory
ls tutorials

# source PATH to use module function
source /cvmfs/soft.computecanada.ca/config/profile/bash.sh

/home/jupyter-mdprieto
reads_directory  results_tutorials  trimmed_reads
For improved speed, add 'usejni=t' to the command line of BBMap tools which support the use of the compiled jni C code.


## Exploring the data

Datasets for this tutorial are availablein a shared folder inside the `tutorials` directory called `raw_reads`. These sequences were downloaded from a biorepository and were produced by Illumina sequencing (75bp paired end reads). This data was collected as part of an outbreak investigation of multidrug resistant _Pseudomonas aeruginosa_ in Switzerland [PMID:34412676](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8376114/). To make it faster and easier to run, we selected only 10 sequenced bacterial samples in raw read format `fastq`. 

The dataset is available in a shared directory (`raw_reads`) so you can move it to the folder you desire using `cp /home/jupyter-mdprieto/reads_directory/* /path/to/target_directory`.

In [17]:
# show content of raw_reads directory
ls /home/jupyter-mdprieto/tutorials/raw_reads

ERR10479510_R1.fastq.gz  ERR10479513_R2.fastq.gz  ERR10479517_R1.fastq.gz
ERR10479510_R2.fastq.gz  ERR10479514_R1.fastq.gz  ERR10479517_R2.fastq.gz
ERR10479511_R1.fastq.gz  ERR10479514_R2.fastq.gz  ERR10479518_R1.fastq.gz
ERR10479511_R2.fastq.gz  ERR10479515_R1.fastq.gz  ERR10479518_R2.fastq.gz
ERR10479512_R1.fastq.gz  ERR10479515_R2.fastq.gz  ERR10479519_R1.fastq.gz
ERR10479512_R2.fastq.gz  ERR10479516_R1.fastq.gz  ERR10479519_R2.fastq.gz
ERR10479513_R1.fastq.gz  ERR10479516_R2.fastq.gz


Also, if the reads are compressed, you could unzip them, although most bioinformatic programs can receive `tar` or `gzip` files as input. 

In [ ]:
# ------------ if you want to decompress the files
#tar -zxf ERR*

The files in our folder have the naming convention `_R1.fastq` or `_R2.fastq`. Because we are using Illumina MiSeq reads, we have pairs of reads per sample and a combination of them represents a single specimen. A similar naming structure is typically used to represent paired end reads (`_1.fastq, _R001.fastq`). More information on the project that provides this data can be found in [Catho et al 2021.](https://pubmed.ncbi.nlm.nih.gov/34412676/)

## Quality control

It is a good practice to organize results in their own directory so you can trace back your analysis steps. You can also use github repositories to synchronize all your results and scripts so other researchers can easily reproduce your results. 

We will create a new directory `results_qc` to put all output data from this analysis pipeline. We can create subdirectories for specific parts of our pipeline if we want. 

<font color='darkred'>_**Notes for compute canada:**_ </font>  
- Compute Canada provides different directories for storage. Jobs cannot be launched from the *HOME* and *PROJECT* directory or any of its subdirectories. The ideal place to run jobs is the *SCRATCH* folder, where you have short term storage of large amounts of data. 
- Once you have final results, these should be moved to your *PROJECT* directory as the *SCRATCH* folder is constantly being cleaned

In [18]:
# creates new directory for results of QC 
mkdir -p /home/jupyter-mdprieto/tutorials/results_qc

Our pipeline for quality control of raw reads includes several steps:

### 1. Seqkit 

In the first step, we use the ***seqkit*** tool to obtain basic statistics from the **.fastq** files. The result is a text file that summarizes basic metrics about the quality and length of sequencing reads. 

In [19]:
# load seqkit parent module and seqkit to our computing environment
module load StdEnv/2020 seqkit/2.3.1

For improved speed, add 'usejni=t' to the command line of BBMap tools which support the use of the compiled jni C code.


In [3]:
# create environment variables to avoid typing PATHs every time
INPUT_DIR="/home/jupyter-mdprieto/tutorials/raw_reads"
OUTPUT_QC="/home/jupyter-mdprieto/tutorials/results_qc"

We can quickly check the results of the `seqkit` command. We see that the text file summarizes the 

In [23]:
# run sequence statistics
seqkit stats $INPUT_DIR/*.fastq.gz > $OUTPUT_QC/seqkit_output.tsv

# check results
head $OUTPUT_QC/seqkit_output.tsv

file                                                                format  type   num_seqs      sum_len  min_len  avg_len  max_len
/home/jupyter-mdprieto/tutorials/raw_reads/ERR10479510_R1.fastq.gz  FASTQ   DNA   1,673,279  386,900,145       35    231.2      251
/home/jupyter-mdprieto/tutorials/raw_reads/ERR10479510_R2.fastq.gz  FASTQ   DNA   1,673,279  387,923,954       35    231.8      251
/home/jupyter-mdprieto/tutorials/raw_reads/ERR10479511_R1.fastq.gz  FASTQ   DNA   1,344,257  305,517,801       35    227.3      251
/home/jupyter-mdprieto/tutorials/raw_reads/ERR10479511_R2.fastq.gz  FASTQ   DNA   1,344,257  307,892,897       35      229      251
/home/jupyter-mdprieto/tutorials/raw_reads/ERR10479512_R1.fastq.gz  FASTQ   DNA   2,091,662  453,818,911       35      217      251
/home/jupyter-mdprieto/tutorials/raw_reads/ERR10479512_R2.fastq.gz  FASTQ   DNA   2,091,662  458,746,957       35    219.3      251
/home/jupyter-mdprieto/tutorials/raw_reads/ERR10479513_R1.fastq.gz  FASTQ   

### 2. fastqc

We will use **fastqc**  to create additional metrics of sequencing including nucleotide distribution, presence of repeats, quality of base calling, GC content and adapter content  
- I input all the files that have the suffix 'fastq.gz' from our reads directory and specify that 5 files will be processed simultaneously (-t 5). The processing should take around 3 minutes for this set of 10 samples. 
- **fastqc** produces an output summary in `zip` and `html` formats for every file entered


In [24]:
# load the tool
module load StdEnv/2020 fastqc/0.11.9

For improved speed, add 'usejni=t' to the command line of BBMap tools which support the use of the compiled jni C code.


In [25]:
fastqc $INPUT_DIR/*.fastq.gz \
    -o $OUTPUT_QC \
    -t 9 \
    --quiet
    # output is saved as individual files in OUTPUT_DIR

Picked up JAVA_TOOL_OPTIONS: -Xmx2g


In [11]:
# resulting files
ls $OUTPUT_QC | head 

ERR10479510_R1_fastqc.html
ERR10479510_R1_fastqc.zip
ERR10479510_R2_fastqc.html
ERR10479510_R2_fastqc.zip
ERR10479511_R1_fastqc.html
ERR10479511_R1_fastqc.zip
ERR10479511_R2_fastqc.html
ERR10479511_R2_fastqc.zip
ERR10479512_R1_fastqc.html
ERR10479512_R1_fastqc.zip


### 3. Trimming reads with BBtools

This step may not be necessary at all, as many genome assemblers are able to remove sequencing adapters from the raw reads before trying to produce a complete genome. In this step we remove the contaminant sequences (adapters) used to amplify the DNA while doing the sequencing experiments; we also remove any reads with poor quality over a moving window of 21 basepairs.

In [3]:
module load StdEnv/2020 bbmap/38.86

For improved speed, add 'usejni=t' to the command line of BBMap tools which support the use of the compiled jni C code.

The following have been reloaded with a version change:
  1) StdEnv/2016.4 => StdEnv/2020           4) intel/2016.4 => intel/2020.1.217
  2) gcccore/.5.4.0 => gcccore/.9.3.0       5) mii/1.1.1 => mii/1.1.2
  3) imkl/11.3.4.258 => imkl/2020.1.217     6) openmpi/2.1.1 => openmpi/4.0.3



The tool includes a text file with commonly used adaptor sequences for Illumina platform. We use this sequences in fasta format as a reference to match and clean our reads. The command we use is `bbduk.sh` and we add several options:

- Specify PATHs to adapter sequences file and to output directory for trimmed reads
- `k=23` specifies the size of the moving window for quality control of reads
- `qtrim=6` removes regions with a score of quality below 6


In [28]:
for i in $(ls $INPUT_DIR/*_R1*)
do
R1=$(basename $i)
R2=$(echo $R1 | sed 's/_R1/_R2/')
bbduk.sh \
    in1=$INPUT_DIR/$R1 in2=$INPUT_DIR/$R2 \
    out1=$OUTPUT_TRIM/$R1 out2=$OUTPUT_TRIM/$R2 \
    ref=$adapters_file \
    k=23 \
    trimq=6 \
    tpe \
    tbo \
    threads=9
done

java -ea -Xmx36987m -Xms36987m -cp /cvmfs/soft.computecanada.ca/easybuild/software/2020/Core/bbmap/38.86/current/ jgi.BBDuk in1=/home/jupyter-mdprieto/tutorials/raw_reads/ERR10479510_R1.fastq.gz in2=/home/jupyter-mdprieto/tutorials/raw_reads/ERR10479510_R2.fastq.gz out1=/home/jupyter-mdprieto/tutorials/trimmed_reads/ERR10479510_R1.fastq.gz out2=/home/jupyter-mdprieto/tutorials/trimmed_reads/ERR10479510_R2.fastq.gz ref=/cvmfs/soft.computecanada.ca/easybuild/software/2020/Core/bbmap/38.86/resources/adapters.fa k=23 trimq=6 tpe tbo threads=9
Picked up JAVA_TOOL_OPTIONS: -Xmx2g
Executing jgi.BBDuk [in1=/home/jupyter-mdprieto/tutorials/raw_reads/ERR10479510_R1.fastq.gz, in2=/home/jupyter-mdprieto/tutorials/raw_reads/ERR10479510_R2.fastq.gz, out1=/home/jupyter-mdprieto/tutorials/trimmed_reads/ERR10479510_R1.fastq.gz, out2=/home/jupyter-mdprieto/tutorials/trimmed_reads/ERR10479510_R2.fastq.gz, ref=/cvmfs/soft.computecanada.ca/easybuild/software/2020/Core/bbmap/38.86/resources/adapters.fa, k=2

### 4. Summarize with multiqc
Finally, another tool (**multiqc**) takes all output summaries of **fastqc** in a directory and creates a nice single HTML output that can be visualized in any internet browser.
The tool is available as a singularity container and can be called using `singularity exec PATH/IMAGE multiqc qc_directory`

In [4]:
singularity exec /home/jupyter-mdprieto/tools/multiqc_1.14.sif multiqc $OUTPUT_QC


  /// ]8;id=202676;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.14

|           multiqc | Search path : /home/jupyter-mdprieto/tutorials/results_qc
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 41/41  0m  
|            fastqc | Found 20 reports
|           multiqc | Compressing plot data
|           multiqc | Report      : multiqc_report.html
|           multiqc | Data        : multiqc_data
|           multiqc | MultiQC complete


In [ ]:
## Reproduce the QC pipeline in Compute Canada

In compute canada, we run intensive commands as part of jobs. In a job, we specify how much memory and processing power we require and the system will automatically asign these values and run our specified commands. 

To do this, we save all of our commands and the instructions for the job in a text file. To do this, we open a text editor (`nano text_filename` or `vim text_filename`) and copy the following commands.
Once the commands are written in the text, we save the file using **Ctrl+X** in `nano` or **Esc + ZZ** in `vim`

```
#!/bin/bash
#SBATCH --account=rrg-whsiao-ab                    # compute canada PI allocation
#SBATCH --mem=25gb                                 # 25 GB of memory
#SBATCH --time=06:00:00                            # approximate time to complete all actions
#SBATCH --job-name="quality_control"               # name of job
#SBATCH --chdir=/scratch/mdprieto/tutorials        # change to directory before start
#SBATCH --cpus-per-task=9                          # number of threads, how many simultaneous tasks
export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

################################ preparation ######################################

# load necessary modules
module load StdEnv/2020 bbmap/38.86 fastqc/0.11.9 seqkit/0.15.0 

# create output directory
mkdir -p /scratch/mdprieto/tutorials/qc_results

# establish path for outputs and input
adapters_file='/cvmfs/soft.computecanada.ca/easybuild/software/2020/Core/bbmap/38.86/resources/adapters.fa'
OUTPUT_QC="/home/jupyter-mdprieto/tutorials/results_qc"
OUTPUT_TRIM="/home/jupyter-mdprieto/tutorials/trimmed_reads"
INPUT_DIR="/home/jupyter-mdprieto/tutorials/raw_reads"

################################## seqkit #########################################

# run seqkit in fastq file and save output in tab separated file
seqkit stats $INPUT_DIR/*.fastq.gz > $OUTPUT_QC/seqkit_output.tsv

################################## fastqc #########################################

# for all files in raw_reads directory
fastqc $INPUT_DIR/*.fastq.gz \
    -o $OUTPUT_QC \
    -t 9 \
    --quiet

################################## multiqc #########################################

# execute from singularity
singularity exec /home/jupyter-mdprieto/tools/multiqc_1.14.sif multiqc $OUTPUT_QC

################################## trimming ########################################

for i in $(ls $INPUT_DIR/*_R1*)
do
R1=$(basename $i)
R2=$(echo $R1 | sed 's/_R1/_R2/')
bbduk.sh \
    in1=$INPUT_DIR/$R1 in2=$INPUT_DIR/$R2 \
    out1=$OUTPUT_TRIM/$R1 out2=$OUTPUT_TRIM/$R2 \
    ref=$adapters_file \
    k=23 trimq=6 tpe tbo \
    threads=9
done

```

<font color='darkred'>_**Notes for compute canada:**_ </font>  
Our pipeline for quality control of raw reads includes several steps:

1. We use the ***seqkit*** tool to obtain basic statistics from the **.fastq** files. The module to load **seqkit** is available in CC, so it can be load using `module load`. 
    - The tool takes `fastq` files as inputs and produces a txt output 
2. We will use **fastqc** which is also available as a module in ComputeCanada. The tool creates an overall summary of different metrics of sequencing including nucleotide distribution, presence of repeats, quality of base calling, GC content and adapter content
    - **fastqc** produces an output summary for every file entered
3. Another tool (**multiqc**) takes all output summaries of **fastqc** in a directory and creates a nice single HTML output that can be visualized in any web browser.
    - Not available in Compute Canada, so we use a singularity container to execute it
4. To have reads ready for assembly, I trimm or remove any PCR adaptors, barcodes or poor quality regions using **BBtools**
    

To make it easier to reproduce, we will open a text file with editor `nano` and save the following code.